In [1]:
import sys
sys.path.append("/home/ubuntu/intelliscope/modules") # go to parent dir

In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization
import tensorflow_addons as tfa
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from intelliscope import instead_data_loaders
from sklearn.metrics import classification_report
import numpy as np
from neptunecontrib.monitoring.metrics import *


In [3]:
PARAMS = {
    'notebook_name':'hybrid01_imbalanced.ipynb', 
    'name':'data_a + imbalanced => Dilated-2 [P22E6]', #(P7E1) 
    'description':'Dilated Conv at first two , batch size 20, 10 epochs ', #[PCG7,ECG1]
    'tags':['imbalanced', 'CASS','Plan 1','c5.12xlarge','Dilated Conv','transfer','RAdam'], 

    'data_balanced':False,    
    
    'pretrained': True, #
    'modelload_pcg_dir':'/home/ubuntu/intelliscope/models/Experiment(PCG-22).h5', # 22 is a good one
    'modelload_ecg_dir':'/home/ubuntu/intelliscope/models/Experiment(ECG-6).h5', #
    'ecg_trainable':True,
    'pcg_trainable':True,
    
    'is_features_normalized':False, 
    'Dilation_Rate':2,
    
    ##########################
    
    'augmentation':0,
    #'opt':tfa.optimizers.RectifiedAdam(lr=1e-3),
    'opt':tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
    'dropout':0.2,    
    'num_epochs': 10,
    
    
    'wavelet':'comb1',    
    'batch_size':20,  
    'validation_split':0.1,
    'test_split': 0.2,
    'dataset':'physionet2016a',
    'pcg/ecg': 'pcg+ecg',
    'pcg_dir':'/home/ubuntu/intelliscope/for_dataloaders/waveletcomb1/dataA/pcg/',  
    'ecg_dir':'/home/ubuntu/intelliscope/for_dataloaders/waveletcomb1/dataA/ecg/', 
    'modelsave_dir':'/home/ubuntu/intelliscope/models',
    }


if PARAMS['data_balanced']==True:
    PARAMS['pcg_dir']+='balanced'
    PARAMS['ecg_dir']+='balanced'
else:
    PARAMS['pcg_dir']+='imbalanced'        
    PARAMS['ecg_dir']+='imbalanced'        
    
    
pcg_dir = PARAMS['pcg_dir']
ecg_dir = PARAMS['ecg_dir']

modelsave_dir  = PARAMS['modelsave_dir']
batch_size     = PARAMS['batch_size']

modelload_pcg_dir = PARAMS['modelload_pcg_dir']
modelload_ecg_dir = PARAMS['modelload_ecg_dir']

In [4]:
print('check ecg vs pcg compability :: all must be True')
print(os.listdir(PARAMS['pcg_dir']+'/train/0') == os.listdir(PARAMS['ecg_dir']+'/train/0'))
print(os.listdir(PARAMS['pcg_dir']+'/train/1') == os.listdir(PARAMS['ecg_dir']+'/train/1'))
print(os.listdir(PARAMS['pcg_dir']+'/val/0')   == os.listdir(PARAMS['ecg_dir']+'/val/0'))
print(os.listdir(PARAMS['pcg_dir']+'/val/1')   == os.listdir(PARAMS['ecg_dir']+'/val/1'))
print(os.listdir(PARAMS['pcg_dir']+'/test/0')  == os.listdir(PARAMS['ecg_dir']+'/test/0'))
print(os.listdir(PARAMS['pcg_dir']+'/test/1')  == os.listdir(PARAMS['ecg_dir']+'/test/1'))

check ecg vs pcg compability :: all must be True
True
True
True
True
True
True


In [5]:
from intelliscope import get_base_model

if PARAMS['pretrained']==True:
    pcg_model=load_model(modelload_pcg_dir)
    ecg_model=load_model(modelload_ecg_dir)
else:
    pcg_model=get_base_model(PARAMS['Dilation_Rate'])
    ecg_model=get_base_model(PARAMS['Dilation_Rate'])
    
pcg_layer=[layer.name for layer in pcg_model.layers][-4]
ecg_layer=[layer.name for layer in ecg_model.layers][-4]

pcg_block= tf.keras.Model(inputs=pcg_model.input, outputs=pcg_model.get_layer(pcg_layer).output)
pcg_block.trainable=PARAMS['ecg_trainable']
pcg_input = tf.keras.layers.Input(shape=(272,462,3))
pcg_features = pcg_block(pcg_input)

ecg_block= tf.keras.Model(inputs=ecg_model.input, outputs=ecg_model.get_layer(ecg_layer).output)
ecg_block.trainable=PARAMS['pcg_trainable']
ecg_input = tf.keras.layers.Input(shape=(272,462,3))
ecg_features = ecg_block(ecg_input)

if PARAMS['is_features_normalized']==True:
    pcg_features = tf.keras.layers.LayerNormalization()(pcg_features)
    ecg_features = tf.keras.layers.LayerNormalization()(ecg_features)
    
concat     = tf.keras.layers.concatenate([pcg_features, ecg_features])
dropout    = tf.keras.layers.Dropout(PARAMS['dropout'])(concat)
dense      = tf.keras.layers.Dense(80, activation='relu')(dropout)
dense_1    = tf.keras.layers.Dense(20, activation='relu')(dense)
output = tf.keras.layers.Dense(2, activation='softmax')(dense_1)

model= tf.keras.Model(inputs=[pcg_input, ecg_input], outputs=[output])

model.compile(PARAMS['opt'],
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.CategoricalAccuracy()])
  

In [6]:
X_train,Y_train = instead_data_loaders(PARAMS,'train')
X_val,Y_val = instead_data_loaders(PARAMS,'val')

X_train_pcg = X_train[:,:,:,0:3]/255.0
X_train_ecg = X_train[:,:,:,3:6]/255.0

X_val_pcg = X_val[:,:,:,0:3]/255.0
X_val_ecg = X_val[:,:,:,3:6]/255.0

history = model.fit([X_train_pcg,X_train_ecg], Y_train,  batch_size=PARAMS['batch_size'], 
                    validation_data = [[X_val_pcg , X_val_ecg], Y_val], 
                    epochs = PARAMS['num_epochs'])



Total  2449
706 Images loaded across 2 Categories. Ignored blanks  8
loading,  1000
loading,  2000
1743 Images loaded across 2 Categories. Ignored blanks  96
2345 Total After removing blanks
Total  340
97 Images loaded across 2 Categories. Ignored blanks  8
2345/2345 [==============================] - 119s 51ms/sample - loss: 0.5713 - categorical_accuracy: 0.7369 - val_loss: 0.4859 - val_categorical_accuracy: 0.7289
Epoch 2/10
2345/2345 [==============================] - 119s 51ms/sample - loss: 0.4367 - categorical_accuracy: 0.8047 - val_loss: 0.4351 - val_categorical_accuracy: 0.8524
Epoch 3/10
2345/2345 [==============================] - 118s 50ms/sample - loss: 0.3542 - categorical_accuracy: 0.8482 - val_loss: 0.3309 - val_categorical_accuracy: 0.8584
Epoch 4/10
2345/2345 [==============================] - 118s 50ms/sample - loss: 0.2244 - categorical_accuracy: 0.9075 - val_loss: 0.3547 - val_categorical_accuracy: 0.8916
Epoch 5/10
2345/2345 [==============================] - 118s 

In [7]:
#!pip install neptune-client
import os
import neptune
NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
os.environ['NEPTUNE_PROJECT']="intelliscope/HYBRID-Imbalanced"

neptune.init('intelliscope/HYBRID-Imbalanced',NEPTUNE_API_TOKEN)
exp = neptune.create_experiment(name=PARAMS['name'],description=PARAMS['description'],params=PARAMS,upload_source_files=PARAMS['notebook_name'],tags=PARAMS['tags'],upload_stdout=True)

model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))
model.save(f'{modelsave_dir}/{str(exp)}.h5')
neptune.log_artifact(f'{modelsave_dir}/{str(exp)}.h5')

for i in range(len(history.history['loss'])):
    neptune.log_metric('loss',history.history['loss'][i])
    neptune.log_metric('val loss',history.history['val_loss'][i])
    neptune.log_metric('categorical_acc',history.history['categorical_accuracy'][i])
    neptune.log_metric('val_categorical_acc',history.history['val_categorical_accuracy'][i])

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/HYBRID-Imbalanced/e/HYBIM-15


In [8]:
X_test,Y_test = instead_data_loaders(PARAMS,'test')

X_test_pcg = X_test[:,:,:,0:3]/255.0
X_test_ecg = X_test[:,:,:,3:6]/255.0

y_pred = model.predict([X_test[:,:,:,0:3],X_test[:,:,:,3:6]], batch_size=X_test.shape[0], verbose=1)

y_pred_bool = np.argmax(y_pred, axis=1)
y_test=np.argmax(Y_test,axis=1)

z = classification_report(y_test, y_pred_bool)
neptune.log_text('classification_report', z)
print(z)


Total  707
199 Images loaded across 2 Categories. Ignored blanks  60
508 Images loaded across 2 Categories. Ignored blanks  9
638 Total After removing blanks
638/638 [==============================] - 2s 3ms/sample
              precision    recall  f1-score   support

           0       0.60      0.54      0.57       139
           1       0.88      0.90      0.89       499

    accuracy                           0.82       638
   macro avg       0.74      0.72      0.73       638
weighted avg       0.82      0.82      0.82       638



In [9]:
threshold = 0.5

log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
log_classification_report(y_test, y_pred[:, 1] > threshold)
log_class_metrics(y_test, y_pred[:, 1] > threshold)
log_class_metrics_by_threshold(y_test, y_pred[:, 1])
log_brier_loss(y_test, y_pred[:, 1])
log_prediction_distribution(y_test, y_pred[:, 1])

log_log_loss(y_test, y_pred)
log_roc_auc(y_test, y_pred)
log_precision_recall_auc(y_test, y_pred)
log_ks_statistic(y_test, y_pred)
log_cumulative_gain(y_test, y_pred)
log_lift_curve(y_test, y_pred)

In [10]:
neptune.stop()